# SETUP
---

## Grab S3 Files

In [1]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

S3_URL = "https://s3-cache.deadlock-api.com"
BUCKET_URL = f"{S3_URL}/db-snapshot"

s3 = boto3.client(
    "s3", config=Config(signature_version=UNSIGNED), endpoint_url=S3_URL
)

match_info_urls = []
match_player_urls = []
heroes_urls = []
items_urls = []

paginator = s3.get_paginator("list_objects_v2")
page_iterator = paginator.paginate(Bucket="db-snapshot", Prefix="public/")
for page in page_iterator:
    for obj in page["Contents"]:
        key = obj["Key"]
        if key.endswith(".parquet"):
            # Match Info
            if key.startswith("public/match_metadata/match_info_"):
                match_info_urls.append(f"{BUCKET_URL}/{key}")
            # Match Player
            if key.startswith("public/match_metadata/match_player_"):
                match_player_urls.append(f"{BUCKET_URL}/{key}")
            # Heroes
            if key.startswith("public/heroes") and key.endswith(".parquet"):
                heroes_urls.append(f"{BUCKET_URL}/{key}")
            # Item
            if key.startswith("public/items"):
                items_urls.append(f"{BUCKET_URL}/{key}")

print("match_info files:", len(match_info_urls))
print("match_player files:", len(match_player_urls))
print("heroes files:", len(heroes_urls))
print("items files:", len(items_urls))

match_info files: 51
match_player files: 51
heroes files: 1
items files: 1


## Settings and Imports

In [2]:
# PHASE 2 + 3 + 4 – COMBINED OPTIMIZED NOTEBOOK
# ------------------------------------------------
# This notebook:
#   1. Builds base tables for the ACTIVE PATCH WINDOW (Phase 2)
#   2. Builds hero-level context + item usage features (Phase 3)
#   3. Builds a sampled training dataset and trains XGBoost models per phase (Phase 4)
#
# Key design change:
#   - All SQL tables are PATCH-AGNOSTIC (no _major / _minor suffixes).
#   - The only place we care about major vs minor is:
#       * computing ACTIVE_PATCH_START (filter window)
#       * naming MODEL_VERSION / deciding whether to rebuild item_assets.
#   - We NEVER build hero_item_training_* as a full cross join.
#   - We sample positives from hero_item_usage_phase and generate negatives in Python.
#   - Heatmap/EDA tables are NOT included here; keep them in a separate notebook.

import duckdb
import pandas as pd
import numpy as np

from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss

from xgboost import XGBClassifier

from deadlock_backend.db import get_connection
from deadlock_backend.config import TRAINING_DB_PATH

# -------------------------
# CONNECT TO DUCKDB
# -------------------------
con = get_connection()
print("Connected to:", TRAINING_DB_PATH)

con.execute("""
INSTALL httpfs;
LOAD httpfs;
SET memory_limit='16GB';
SET temp_directory='../data/duckdb/tmp/';
SET threads=8;
SET enable_progress_bar=true;
PRAGMA enable_optimizer;
PRAGMA preserve_insertion_order = FALSE;
""")

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

Connected to: /home/karan/dev/deadlock/deadlock-backend/data/duckdb/deadlock_db.duckdb


## Getting Patches

In [ ]:
import requests
from datetime import datetime, timezone

# -------------------------------
# 1. Fetch major patches
# -------------------------------
def fetch_major_patches():
    url = "https://api.deadlock-api.com/v1/patches/big-days"
    res = requests.get(url, timeout=10)
    res.raise_for_status()
    dates = res.json()
    major_patches = sorted(
        [datetime.fromisoformat(ts.replace("Z", "+00:00")) for ts in dates],
        reverse=True  # newest → oldest
    )
    return major_patches

# -------------------------------
# 2. Fetch minor patches
# -------------------------------
def fetch_minor_patches():
    url = "https://api.deadlock-api.com/v1/patches"
    res = requests.get(url, timeout=10)
    res.raise_for_status()
    patches = res.json()  # list of patch objects

    minor_patches = []
    for p in patches:
        title = p.get("title", "")
        if not title:
            continue

        # Example: "11-21-2025 Update" -> "11-21-2025"
        date_str = title.split()[0]

        try:
            # Parse MM-DD-YYYY into a datetime and make it UTC
            dt = datetime.strptime(date_str, "%m-%d-%Y").replace(tzinfo=timezone.utc)
            minor_patches.append(dt)
        except ValueError:
            continue

    minor_patches.sort(reverse=True)
    return minor_patches


# -------------------------------
# 3. Get patch boundaries for pipeline
# -------------------------------
major_patches = fetch_major_patches()
minor_patches = fetch_minor_patches()

latest_major_patch = major_patches[0]
latest_minor_patch = minor_patches[0]

print("Latest Major Patch:", latest_major_patch)
print("Latest Minor Patch:", latest_minor_patch)

# Decide which patch is the active patch for training
if latest_major_patch >= latest_minor_patch:
    ACTIVE_PATCH_TYPE = "major"
    ACTIVE_PATCH_START = latest_major_patch
else:
    ACTIVE_PATCH_TYPE = "minor"
    ACTIVE_PATCH_START = latest_minor_patch

print(f"\n=== ACTIVE PATCH FOR TRAINING ===")
print(f"Type     : {ACTIVE_PATCH_TYPE}")
print(f"Start TS : {ACTIVE_PATCH_START}")


Latest Major Patch: 2025-08-18 20:43:52+00:00
Latest Minor Patch: 2025-12-29 00:00:00+00:00

=== ACTIVE PATCH FOR TRAINING ===
Type     : minor
Start TS : 2025-12-29 00:00:00+00:00


## Globals

In [ ]:
from pathlib import Path
from deadlock_backend.config import MODELS_ROOT

# -------------------------
# PATH + PATCH CONFIG
# -------------------------
ACTIVE_PATCH = ACTIVE_PATCH_TYPE

MODEL_VERSION = f"{ACTIVE_PATCH_TYPE}_{ACTIVE_PATCH_START:%Y%m%d}"
MODELS_DIR = MODELS_ROOT / MODEL_VERSION
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print("MODEL_VERSION:", MODEL_VERSION)
print("MODELS_DIR:", MODELS_DIR.resolve())

PHASES = ["early", "mid", "late", "very_late"]

LAMBDA_BY_PHASE_DEFAULT = {
    "early": 0.01,
    "mid":   0.03,
    "late":  0.05,
    "very_late": 0.02,
}

SLOTS_PER_PHASE_DEFAULT = {
    "early": 4,
    "mid": 4,
    "late": 4,
    "very_late": 3,
}

TOP_N_DEFAULT = 10

# Phantom+ threshold
PHANTOM_TIER = 9

# TRAINING CONFIG
# minimum matches in active window to train
MIN_MATCHES_WINDOW = 10_000 
CAN_TRAIN = True

MAX_POS_ROWS = {
    "major": 2_000_000,
    "minor": 800_000,
}
MAX_POS_ROWS_ACTIVE = MAX_POS_ROWS[ACTIVE_PATCH]

NEG_PER_POS = 3
RANDOM_SEED = 42

LABEL_COL = "item_bought"

NUMERIC_FEATURES = [
    "duration_s",
    "souls_9m",
    "cs_9m",
    "kills_9m",
    "lane_adv_signed",

    "team0_tier",
    "team1_tier",

    "item_global_wr",
    "hero_item_wr",

    "synergy_avg",
    "synergy_max",
    "synergy_sum",
    "synergy_strong_count",

    "counter_avg",
    "counter_max",
    "counter_sum",
    "counter_hard_count",

    "avg_soul_diff",
    "avg_souls_raw",
    "lane_tower_rate",
]

CATEGORICAL_FEATURES = [
    "hero_id",
    "item_id",
    "team",
    "assigned_lane",
    "lane_opponent",
]

METADATA_COLS = ["match_id", "hero_id", "item_id", "phase"]

ALL_FEATURES = NUMERIC_FEATURES + CATEGORICAL_FEATURES

MODEL_VERSION: minor_20251229
MODELS_DIR: /home/karan/dev/deadlock/deadlock-backend/models/minor_20251229


# Support Tables
---

## Heroes and Items

In [5]:
# =========================
# SHARED DIMENSION TABLES
# =========================

import requests

# Heroes
con.execute(f"""
CREATE OR REPLACE TABLE heroes AS
SELECT *
FROM read_parquet({heroes_urls})
""")

# Items (cost > 0 only)
con.execute(f"""
CREATE OR REPLACE TABLE shop_items AS
SELECT *
FROM read_parquet({items_urls})
WHERE cost > 0
""")

print(con.execute("SELECT COUNT(*) AS n_heroes FROM heroes").df())
print(con.execute("SELECT COUNT(*) AS n_items FROM shop_items").df())

   n_heroes
0        32
   n_items
0      151


## Item Images

In [ ]:
# -----------------------------------------------------------------------------
# ITEM_ASSETS: maps item_id → item name + shop image URLs
# -----------------------------------------------------------------------------
# Only rebuild on MAJOR patch to save API calls
REBUILD_ITEM_ASSETS = (ACTIVE_PATCH_TYPE == "major")

if REBUILD_ITEM_ASSETS:
    item_ids = con.execute("SELECT DISTINCT id FROM shop_items").df()["id"].tolist()
    rows = []
    for item_id in item_ids:
        url = f"https://assets.deadlock-api.com/v2/items/{item_id}?language=english&client_version=5983"
        data = requests.get(url).json()
        rows.append((
            item_id,
            data["name"],
            data.get("shop_image"),
            data.get("shop_image_webp"),
            data.get("item_slot_type"),
        ))

    df_item_assets = pd.DataFrame(rows, columns=["item_id", "name", "shop_image", "shop_image_webp", "item_slot_type"])
    con.register("df_item_assets", df_item_assets)

    con.execute("""
    CREATE OR REPLACE TABLE item_assets AS
    SELECT * FROM df_item_assets
    """)

    print(con.execute("SELECT COUNT(*) AS n_item_assets FROM item_assets").df())
else:
    print("Skipping item_assets rebuild for minor patch; using existing table.")


Skipping item_assets rebuild for minor patch; using existing table.


In [7]:
con.execute("SELECT * FROM item_assets LIMIT 5").df()

,item_id,name,shop_image,shop_image_webp,item_slot_type
0,343572757,Spirit Burn,https://assets-bucket.deadlock-api.com/assets-...,https://assets-bucket.deadlock-api.com/assets-...,spirit
1,393974127,Slowing Bullets,https://assets-bucket.deadlock-api.com/assets-...,https://assets-bucket.deadlock-api.com/assets-...,weapon
2,395944548,Torment Pulse,https://assets-bucket.deadlock-api.com/assets-...,https://assets-bucket.deadlock-api.com/assets-...,spirit
3,630839635,Echo Shard,https://assets-bucket.deadlock-api.com/assets-...,https://assets-bucket.deadlock-api.com/assets-...,spirit
4,754480263,Mystic Expansion,https://assets-bucket.deadlock-api.com/assets-...,https://assets-bucket.deadlock-api.com/assets-...,spirit


## Hero Images

In [ ]:
# -----------------------------------------------------------------------------
# HERO_ASSETS: maps hero_id → hero name + hero image URLs
# -----------------------------------------------------------------------------
# Only rebuild on MAJOR patch to save API calls
REBUILD_HERO_ASSETS = (ACTIVE_PATCH_TYPE == "major")

if REBUILD_HERO_ASSETS:
    hero_ids = con.execute("SELECT DISTINCT id FROM heroes").df()["id"].tolist()
    rows = []
    for hero_id in hero_ids:
        url = f"https://assets.deadlock-api.com/v2/heroes/{hero_id}?language=english&client_version=6008"
        data = requests.get(url).json()
        rows.append((
            hero_id,
            data["name"],
            data["images"].get("icon_hero_card"),
            data["images"].get("icon_hero_card_webp")
        ))

    df_hero_assets = pd.DataFrame(rows, columns=["hero_id", "name", "icon_hero_card", "icon_hero_card_webp"])
    con.register("df_hero_assets", df_hero_assets)

    con.execute("""
    CREATE OR REPLACE TABLE hero_assets AS
    SELECT * FROM df_hero_assets
    """)

    print(con.execute("SELECT COUNT(*) AS n_hero_assets FROM hero_assets").df())
else:
    print("Skipping hero_assets rebuild for minor patch; using existing table.")

Skipping hero_assets rebuild for minor patch; using existing table.


# Phase 2
---

## Helper Functions

### Build `match_info`

In [9]:
## Build `match_info` (patch-agnostic table name)
def build_match_info():
    """
    Build match_info for the ACTIVE_PATCH window (major or minor),
    filtered by Phantom+ average badge and with rank metadata joined in.
    """
    start_ts = ACTIVE_PATCH_START
    table = "match_info"

    print(f"Building {table} for ACTIVE_PATCH={ACTIVE_PATCH_TYPE} starting {start_ts} ...")

    query = f"""
    CREATE OR REPLACE TABLE {table} AS
    WITH base AS (
        SELECT *
        FROM read_parquet({match_info_urls})
        WHERE start_time >= TIMESTAMP '{start_ts}'
          AND average_badge_team0 IS NOT NULL
          AND average_badge_team1 IS NOT NULL
          AND average_badge_team0 > 0
          AND average_badge_team1 > 0
    ),

    ranked AS (
        SELECT
            b.*,
            r0.tier       AS team0_tier,
            r0.subrank    AS team0_subrank,
            r0.rank_name  AS team0_rank_name,
            r0.rank_full  AS team0_rank_full,
            r1.tier       AS team1_tier,
            r1.subrank    AS team1_subrank,
            r1.rank_name  AS team1_rank_name,
            r1.rank_full  AS team1_rank_full
        FROM base b
        LEFT JOIN rank_lookup r0
            ON b.average_badge_team0 = r0.badge
        LEFT JOIN rank_lookup r1
            ON b.average_badge_team1 = r1.badge
    ),

    filtered AS (
        SELECT *
        FROM ranked
        WHERE team0_tier IS NOT NULL
          AND team1_tier IS NOT NULL
          AND team0_tier >= {PHANTOM_TIER}
          AND team1_tier >= {PHANTOM_TIER}
    )

    SELECT * FROM filtered;
    """
    con.execute(query)
    con.execute(f"ANALYZE {table}")
    print(con.execute(f"SELECT COUNT(*) AS n FROM {table}").df())


### Build `match_player`

In [10]:
def build_match_player():
    mi_table = "match_info"
    mp_table = "match_player"

    print(f"Building {mp_table} ...")

    con.execute(f"ANALYZE {mi_table}")
    con.execute(f"CREATE OR REPLACE TEMP TABLE tmp_match_ids AS SELECT match_id FROM {mi_table};")
    con.execute(f"CREATE OR REPLACE TEMP TABLE tmp_heroes AS SELECT id AS hero_id FROM heroes;")

    query = f"""
    CREATE OR REPLACE TABLE {mp_table} AS
    WITH base AS (
        SELECT mp.*
        FROM read_parquet({match_player_urls}) mp
        JOIN tmp_match_ids t USING (match_id)
    ),
    clean AS (
        SELECT b.*
        FROM base b
        JOIN tmp_heroes h ON h.hero_id = b.hero_id
        WHERE b.team IN ('Team0','Team1')
          AND b.assigned_lane IS NOT NULL
    ),
    valid_matches AS (
        SELECT match_id
        FROM clean
        GROUP BY match_id
        HAVING COUNT(*) = 12
    ),
    ranked AS (
        SELECT
            c.*,
            CASE WHEN c.team='Team0' THEN mi.team0_tier  ELSE mi.team1_tier  END AS tier,
            CASE WHEN c.team='Team0' THEN mi.team0_subrank ELSE mi.team1_subrank END AS subrank
        FROM clean c
        JOIN valid_matches v USING (match_id)
        JOIN {mi_table} mi USING (match_id)
    )
    SELECT * FROM ranked;
    """
    con.execute(query)

    # Filter match_info down to 12-player matches only
    con.execute(f"""
    CREATE OR REPLACE TABLE {mi_table} AS
    SELECT mi.*
    FROM {mi_table} mi
    JOIN (SELECT DISTINCT match_id FROM {mp_table}) v USING (match_id);
    """)

    print(con.execute(f"SELECT COUNT(*) AS n_players, COUNT(DISTINCT match_id) AS n_matches FROM {mp_table}").df())

### Build `hero_item_usage`

In [11]:
def build_hero_item_usage():
    mp_table = "match_player"
    hiu_table = "hero_item_usage"

    print(f"Building {hiu_table} ...")

    con.execute("ANALYZE shop_items")
    con.execute(f"ANALYZE {mp_table}")

    query = f"""
    CREATE OR REPLACE TABLE {hiu_table} AS
    SELECT
        mp.match_id,
        mp.hero_id,
        item_id,
        mp."items.game_time_s"[idx] AS time_s,
        mp.won
    FROM {mp_table} mp,
         UNNEST(mp."items.item_id") WITH ORDINALITY AS t(item_id, idx)
    JOIN shop_items si ON si.id = item_id
    WHERE mp.hero_id IS NOT NULL;
    """
    con.execute(query)

    print(con.execute(f"""
        SELECT COUNT(*) AS n_rows, 
               COUNT(DISTINCT match_id) AS n_matches,
               COUNT(DISTINCT hero_id) AS n_heroes
        FROM {hiu_table}
    """).df())


### Build `team_comp` and `enemy_team_comp`

In [12]:
def build_team_comps():
    mp_table = "match_player"
    tc_table = "team_comp"
    etc_table = "enemy_team_comp"

    print(f"Building {tc_table} and {etc_table} ...")

    query_tc = f"""
    CREATE OR REPLACE TABLE {tc_table} AS
    WITH team_comps AS (
        SELECT
            match_id,
            team,
            ARRAY_AGG(hero_id ORDER BY hero_id) AS heroes
        FROM {mp_table}
        GROUP BY match_id, team
    )
    SELECT
        mp.match_id,
        mp.hero_id,
        mp.team,
        tc.heroes AS ally_heroes
    FROM {mp_table} mp
    JOIN team_comps tc
      ON mp.match_id = tc.match_id
     AND mp.team = tc.team;
    """

    query_etc = f"""
    CREATE OR REPLACE TABLE {etc_table} AS
    WITH enemy_comps AS (
        SELECT
            match_id,
            team,
            ARRAY_AGG(hero_id ORDER BY hero_id) AS heroes
        FROM {mp_table}
        GROUP BY match_id, team
    )
    SELECT
        mp.match_id,
        mp.hero_id,
        mp.team,
        ec.heroes AS enemy_heroes
    FROM {mp_table} mp
    JOIN enemy_comps ec
      ON mp.match_id = ec.match_id
     AND mp.team != ec.team;
    """

    con.execute(query_tc)
    con.execute(query_etc)

    print(con.execute(f"SELECT COUNT(*) AS n FROM {tc_table}").df())
    print(con.execute(f"SELECT COUNT(*) AS n FROM {etc_table}").df())

### Build `synergy` and `counter`

In [13]:
def build_synergy_and_counter():
    mp_table = "match_player"
    synergy_table = "hero_synergy"
    counter_table = "hero_counter"

    print(f"Building {synergy_table} and {counter_table} ...")

    # Synergy (heroes on same team)
    query_synergy = f"""
    CREATE OR REPLACE TABLE {synergy_table} AS
    WITH paired AS (
        SELECT
            LEAST(a.hero_id, b.hero_id)    AS hero1,
            GREATEST(a.hero_id, b.hero_id) AS hero2,
            a.won AS won
        FROM {mp_table} a
        JOIN {mp_table} b
          ON a.match_id = b.match_id
         AND a.team = b.team
         AND a.hero_id < b.hero_id
    ),
    named AS (
        SELECT
            p.hero1,
            h1.name AS hero1_name,
            p.hero2,
            h2.name AS hero2_name,
            p.won
        FROM paired p
        JOIN heroes h1 ON h1.id = p.hero1
        JOIN heroes h2 ON h2.id = p.hero2
    )
    SELECT
        hero1,
        hero1_name,
        hero2,
        hero2_name,
        COUNT(*) AS matches,
        SUM(CASE WHEN won THEN 1 ELSE 0 END) AS wins,
        ROUND(100 * (SUM(CASE WHEN won THEN 1 ELSE 0 END) / COUNT(*)), 2) AS winrate
    FROM named 
    GROUP BY hero1_name, hero2_name, hero1, hero2;
    """
    con.execute(query_synergy)

    # Counter (heroes in same lane, opposite team)
    query_counter = f"""
    CREATE OR REPLACE TABLE {counter_table} AS
    WITH paired AS (
        SELECT
            a.hero_id AS hero,
            b.hero_id AS enemy,
            a.won AS won
        FROM {mp_table} a
        JOIN {mp_table} b
          ON a.match_id = b.match_id
         AND a.team != b.team
         AND a.assigned_lane = b.assigned_lane
         AND a.hero_id != b.hero_id
    ),
    named AS (
        SELECT
            hero,
            h.name AS hero_name,
            enemy,
            e.name AS enemy_name,
            won
        FROM paired p
        JOIN heroes h ON h.id = p.hero
        JOIN heroes e ON e.id = p.enemy
    )
    SELECT
        hero,
        hero_name,
        enemy,
        enemy_name,
        COUNT(*) AS matches,
        SUM(CASE WHEN won THEN 1 ELSE 0 END) AS wins,
        ROUND(100 * (SUM(CASE WHEN won THEN 1 ELSE 0 END) / COUNT(*)), 2) AS winrate
    FROM named
    GROUP BY hero, enemy, hero_name, enemy_name;
    """
    con.execute(query_counter)

    print(con.execute(f"SELECT COUNT(*) AS n FROM {synergy_table}").df())
    print(con.execute(f"SELECT COUNT(*) AS n FROM {counter_table}").df())

### Build `lane_advantage` and `matchups`

In [14]:
def build_lane_advantage_and_matchups():
    mp_table   = "match_player"
    mi_table   = "match_info"
    lane_table = "lane_advantage"
    snap_table = "hero_lane_snap_9"
    hvh_table  = "hero_vs_hero_lane"
    soul_table = "hero_soul_matchup"

    print("Building lane advantage + lane matchup tables ...")

    # Lane advantage
    query_lane = f"""
    CREATE OR REPLACE TABLE {lane_table} AS
    WITH
    snapshots AS (
        SELECT
            mp.match_id,
            mp.hero_id,
            mp.team,
            mp.assigned_lane,
            UNNEST(mp."stats.time_stamp_s") AS ts,
            UNNEST(mp."stats.net_worth") AS souls,
            UNNEST(mp."stats.gold_lane_creep_orbs") AS cs,
            UNNEST(mp."stats.kills") AS kills
        FROM {mp_table} mp
    ),
    snapshots_ranked AS (
        SELECT *,
            ROW_NUMBER() OVER (
                PARTITION BY match_id, hero_id
                ORDER BY ABS(CAST(ts AS INT) - 540)
            ) AS rn
        FROM snapshots
    ),
    snap_9 AS (
        SELECT
            match_id,
            hero_id,
            team,
            assigned_lane,
            souls AS souls_9m,
            cs AS cs_9m,
            kills AS kills_9m
        FROM snapshots_ranked
        WHERE rn = 1
    ),
    tower_events AS (
        SELECT
            mi.match_id,
            t.obj,
            t.idx,
            mi."objectives.team"[t.idx]             AS team,
            mi."objectives.destroyed_time_s"[t.idx] AS destroyed_time,
            CASE t.obj
                WHEN 'Tier1Lane1' THEN 1
                WHEN 'Tier1Lane3' THEN 2
                WHEN 'Tier1Lane4' THEN 3
                ELSE NULL
            END AS lane
        FROM {mi_table} mi,
             UNNEST(mi."objectives.team_objective") 
             WITH ORDINALITY AS t(obj, idx)
        WHERE t.obj IN ('Tier1Lane1','Tier1Lane3','Tier1Lane4')
    ),
    ranked_tower AS (
        SELECT
            match_id,
            lane,
            team,
            destroyed_time,
            ROW_NUMBER() OVER (
                PARTITION BY match_id, lane
                ORDER BY destroyed_time
            ) AS rn
        FROM tower_events
        WHERE lane IS NOT NULL
          AND destroyed_time > 0
    ),
    tower_firstblood AS (
        SELECT
            match_id,
            lane,
            team AS tower_team,
            destroyed_time
        FROM ranked_tower
        WHERE rn = 1
    ),
    lane_team AS (
        SELECT
            match_id,
            CAST(ROUND(assigned_lane/2) AS INT) AS lane,
            team,
            SUM(souls_9m) AS souls_duo,
            SUM(cs_9m)    AS cs_duo,
            SUM(kills_9m) AS kills_duo
        FROM snap_9
        GROUP BY match_id, assigned_lane, team
    ),
    lane_duo AS (
        SELECT
            t0.match_id,
            t0.lane,
            t0.team AS team0,
            t1.team AS team1,
            t0.souls_duo AS souls_duo_team0,
            t1.souls_duo AS souls_duo_team1,
            t0.cs_duo    AS cs_duo_team0,
            t1.cs_duo    AS cs_duo_team1,
            t0.kills_duo AS kills_duo_team0,
            t1.kills_duo AS kills_duo_team1
        FROM lane_team t0
        JOIN lane_team t1
          ON t0.match_id = t1.match_id
         AND t0.lane = t1.lane
         AND t0.team != t1.team
        WHERE t0.team = 'Team0'
    ),
    lane_score AS (
        SELECT
            d.match_id,
            d.lane,
            CASE 
                WHEN fb.tower_team = d.team0 THEN 1
                WHEN fb.tower_team = d.team1 THEN -1
                ELSE 0
            END AS tower_firstblood,
            (d.souls_duo_team0 - d.souls_duo_team1) / 500.0 AS souls_diff,
            (d.cs_duo_team0    - d.cs_duo_team1)    / 100.0 AS cs_diff,
            (d.kills_duo_team0 - d.kills_duo_team1)         AS kills_diff,
            3 * CASE 
                    WHEN fb.tower_team = d.team0 THEN 1
                    WHEN fb.tower_team = d.team1 THEN -1
                    ELSE 0
                END
          + 2 * ((d.souls_duo_team0 - d.souls_duo_team1) / 500.0)
          + 1 * (d.kills_duo_team0 - d.kills_duo_team1)
          + 1 * ((d.cs_duo_team0 - d.cs_duo_team1) / 100.0)
            AS lane_advantage_score
        FROM lane_duo d
        LEFT JOIN tower_firstblood fb
          ON fb.match_id = d.match_id
         AND fb.lane = d.lane
    )
    SELECT * FROM lane_score;
    """
    con.execute(query_lane)

    # Per-hero lane snapshot table
    query_snap = f"""
    CREATE OR REPLACE TABLE {snap_table} AS
    WITH snapshots AS (
        SELECT
            mp.match_id,
            mp.hero_id,
            mp.team,
            mp.assigned_lane,
            UNNEST(mp."stats.time_stamp_s")           AS ts,
            UNNEST(mp."stats.net_worth")             AS souls,
            UNNEST(mp."stats.gold_lane_creep_orbs")  AS cs,
            UNNEST(mp."stats.kills")                 AS kills
        FROM {mp_table} mp
    ),
    ranked AS (
        SELECT *,
            ROW_NUMBER() OVER (
                PARTITION BY match_id, hero_id
                ORDER BY ABS(CAST(ts AS INT) - 540)
            ) AS rn
        FROM snapshots
    )
    SELECT
        match_id,
        hero_id,
        team,
        assigned_lane,
        CAST(ROUND(assigned_lane/2) AS INT) AS lane,
        souls AS souls_9m,
        cs    AS cs_9m,
        kills AS kills_9m
    FROM ranked
    WHERE rn = 1;
    """
    con.execute(query_snap)

    # Hero vs hero lane table
    query_hvh = f"""
    CREATE OR REPLACE TABLE {hvh_table} AS
    WITH h AS (
        SELECT * FROM {snap_table}
    ),
    ls AS (
        SELECT * FROM {lane_table}
    ),
    pairs AS (
        SELECT
            h0.match_id,
            h0.lane,
            h0.hero_id  AS hero0,
            h1.hero_id  AS hero1,
            h0.souls_9m AS souls0_9m,
            h1.souls_9m AS souls1_9m,
            ls.lane_advantage_score,
            ls.tower_firstblood
        FROM h h0
        JOIN h h1
          ON h0.match_id = h1.match_id
         AND h0.lane     = h1.lane
         AND h0.team     = 'Team0'
         AND h1.team     = 'Team1'
        JOIN ls
          ON ls.match_id = h0.match_id
         AND ls.lane     = h0.lane
    )
    SELECT
        match_id,
        lane,
        hero0        AS hero,
        hero1        AS opponent,
        lane_advantage_score AS advantage,
        souls0_9m    AS hero_souls_9m,
        CASE 
            WHEN tower_firstblood = 1 THEN 1
            WHEN tower_firstblood = -1 THEN 0
            ELSE NULL
        END          AS got_tower
    FROM pairs
    UNION ALL
    SELECT
        match_id,
        lane,
        hero1        AS hero,
        hero0        AS opponent,
        -lane_advantage_score AS advantage,
        souls1_9m    AS hero_souls_9m,
        CASE 
            WHEN tower_firstblood = -1 THEN 1
            WHEN tower_firstblood = 1  THEN 0
            ELSE NULL
        END          AS got_tower
    FROM pairs;
    """
    con.execute(query_hvh)

    # Hero soul matchup table
    query_soul = f"""
    CREATE OR REPLACE TABLE {soul_table} AS
    WITH hvh AS (
        SELECT
            match_id,
            lane,
            hero,
            opponent,
            hero_souls_9m,
            got_tower
        FROM {hvh_table}
    ),
    paired AS (
        SELECT
            a.match_id,
            a.lane,
            a.hero,
            a.opponent,
            a.hero_souls_9m AS hero_souls_9m,
            b.hero_souls_9m AS opponent_souls_9m,
            a.got_tower
        FROM hvh a
        JOIN hvh b
          ON a.match_id = b.match_id
         AND a.lane = b.lane
         AND a.hero = b.opponent
         AND a.opponent = b.hero
    )
    SELECT
        hero,
        opponent,
        AVG(CAST(hero_souls_9m AS INT) - CAST(opponent_souls_9m AS INT)) AS avg_soul_diff,
        AVG(CAST(hero_souls_9m AS INT))                                 AS avg_souls_raw,
        AVG(got_tower)                                                  AS tower_rate
    FROM paired
    GROUP BY hero, opponent;
    """
    con.execute(query_soul)

    print(con.execute(f"SELECT COUNT(*) AS n_lanes FROM {lane_table}").df())
    print(con.execute(f"SELECT COUNT(*) AS n_hvh   FROM {hvh_table}").df())
    print(con.execute(f"SELECT COUNT(*) AS n_hsoul FROM {soul_table}").df())

### Build `hero_item_winrate`

In [15]:
def build_hero_item_winrate(min_matches: int = 20):
    hiu_table = "hero_item_usage"
    hiw_table = "hero_item_winrate"

    print(f"Building {hiw_table} ...")

    query = f"""
    CREATE OR REPLACE TABLE {hiw_table} AS
    WITH base AS (
        SELECT 
            hero_id,
            item_id,
            SUM(CASE WHEN won THEN 1 ELSE 0 END) AS wins,
            COUNT(DISTINCT match_id) AS matches
        FROM {hiu_table}
        GROUP BY hero_id, item_id
        HAVING COUNT(DISTINCT match_id) >= {min_matches}
    ),
    computed AS (
        SELECT
            hero_id,
            item_id,
            wins,
            matches,
            ROUND(wins * 100.0 / matches, 2) AS winrate,
            ROUND(
                100.0 * (wins + 150.0) / (matches + 300.0),
                2
            ) AS smoothed_wr,
            (
                (
                    (wins * 1.0 / matches) 
                    + (1.96 * 1.96) / (2 * matches)
                    - 1.96 * sqrt(
                        (wins * 1.0 / matches) * (1 - (wins * 1.0 / matches)) / matches
                        + (1.96 * 1.96) / (4 * matches * matches)
                    )
                )
                /
                (1 + (1.96 * 1.96) / matches)
            ) * 100.0 AS wilson_lower
        FROM base
    )
    SELECT * FROM computed;
    """
    con.execute(query)

    print(con.execute(f"SELECT COUNT(*) AS n FROM {hiw_table}").df())

### Build `item_transition_stats`

In [16]:
def build_item_transition_stats(min_transitions: int = 20):
    """
    For each hero, compute how often item B follows item A in the purchase
    sequence, and normalize to a transition probability.

    Output table: item_transition_stats
      - hero_id
      - item_current
      - item_next
      - trans_count
      - trans_prob   (P(item_next | hero, item_current))
    """
    hiu_table = "hero_item_usage"
    its_table = "item_transition_stats"

    print(f"Building {its_table} ...")

    query = f"""
    CREATE OR REPLACE TABLE {its_table} AS
    WITH seq AS (
        SELECT
            match_id,
            hero_id,
            item_id,
            time_s,
            LEAD(item_id) OVER (
                PARTITION BY match_id, hero_id
                ORDER BY time_s
            ) AS next_item
        FROM {hiu_table}
    ),
    filtered AS (
        SELECT
            hero_id,
            item_id      AS item_current,
            next_item    AS item_next
        FROM seq
        WHERE next_item IS NOT NULL
    ),
    agg AS (
        SELECT
            hero_id,
            item_current,
            item_next,
            COUNT(*) AS trans_count
        FROM filtered
        GROUP BY hero_id, item_current, item_next
        HAVING COUNT(*) >= {min_transitions}
    )
    SELECT
        hero_id,
        item_current,
        item_next,
        trans_count,
        trans_count * 1.0 /
          SUM(trans_count) OVER (PARTITION BY hero_id, item_current) AS trans_prob
    FROM agg;
    """
    con.execute(query)
    print(con.execute(f"SELECT COUNT(*) AS n FROM {its_table}").df())

## Build

In [ ]:
print("\n==============================")
print(f"BUILDING PHASE 2 BASE TABLES – ACTIVE PATCH = {ACTIVE_PATCH_TYPE.upper()}")
print("==============================")

# 1) Build match_info for the active patch window
build_match_info()

# 2) Check how many matches we have in this window
active_match_count = con.execute("""
    SELECT COUNT(*) AS n_matches
    FROM match_info
""").df()["n_matches"].iloc[0]

print(f"[ACTIVE] Matches in active window: {active_match_count}")

if active_match_count < MIN_MATCHES_WINDOW:
    print(f"WARNING: only {active_match_count} matches (< {MIN_MATCHES_WINDOW}).")
    print("Setting CAN_TRAIN = False; models will NOT be retrained this run.")
    CAN_TRAIN = False
else:
    CAN_TRAIN = True

# 3) Build downstream Phase 2 tables
if CAN_TRAIN:
    build_match_player()
    build_hero_item_usage()
    build_team_comps()
    build_synergy_and_counter()
    build_lane_advantage_and_matchups()
    build_hero_item_winrate(min_matches=20)
    build_item_transition_stats(min_transitions=20)


BUILDING PHASE 2 BASE TABLES – ACTIVE PATCH = MINOR
Building match_info for ACTIVE_PATCH=minor starting 2025-12-29 00:00:00+00:00 ...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

       n
0  60121
[ACTIVE] Matches in active window: 60121
Building match_player ...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   n_players  n_matches
0     721452      60121
Building hero_item_usage ...


     n_rows  n_matches  n_heroes
0  11592438      60120        32
Building team_comp and enemy_team_comp ...


        n
0  721452
        n
0  721452
Building hero_synergy and hero_counter ...
     n
0  496
     n
0  992
Building lane advantage + lane matchup tables ...


   n_lanes
0   180363
     n_hvh
0  1442904
   n_hsoul
0      992
Building hero_item_winrate ...
      n
0  4283
Building item_transition_stats ...


       n
0  56537


# Phase 3
---

## Helper Functions

### Build `hero_context`

In [18]:
## Build `hero_context`
def build_hero_context():
    mp_table   = "match_player"
    mi_table   = "match_info"
    snap_table = "hero_lane_snap_9"
    lane_table = "lane_advantage"
    tc_table   = "team_comp"
    etc_table  = "enemy_team_comp"
    ctx_table  = "hero_context"

    print(f"Building {ctx_table} ...")

    query = f"""
    CREATE OR REPLACE TABLE {ctx_table} AS
    SELECT
        mp.match_id,
        mp.hero_id,
        mp.account_id,
        mp.team,
        mp.won,
        mp.assigned_lane,

        mi.duration_s,
        mi.team0_tier,
        mi.team0_subrank,
        mi.team1_tier,
        mi.team1_subrank,

        hls.souls_9m,
        hls.cs_9m,
        hls.kills_9m,

        ls.lane_advantage_score,

        tc.ally_heroes,
        etc.enemy_heroes

    FROM {mp_table} mp
    JOIN {mi_table} mi USING (match_id)

    LEFT JOIN {snap_table} hls
           ON hls.match_id = mp.match_id
          AND hls.hero_id = mp.hero_id

    LEFT JOIN {lane_table} ls
           ON ls.match_id = mp.match_id
          AND ls.lane = CAST(ROUND(mp.assigned_lane/2) AS INT)

    LEFT JOIN {tc_table} tc
           ON tc.match_id = mp.match_id
          AND tc.hero_id  = mp.hero_id

    LEFT JOIN {etc_table} etc
           ON etc.match_id = mp.match_id
          AND etc.hero_id  = mp.hero_id;
    """
    con.execute(query)
    print(con.execute(f"SELECT COUNT(*) AS n FROM {ctx_table}").df())

### Build `hero_synergy_agg`

In [19]:
def build_hero_synergy_agg():
    ctx_table = "hero_context"
    base_sy   = "hero_synergy"
    agg_table = "hero_synergy_agg"

    print(f"Building {agg_table} ...")

    query = f"""
    CREATE OR REPLACE TABLE {agg_table} AS
    WITH exploded AS (
        SELECT
            hc.match_id,
            hc.hero_id,
            ally AS ally_id,
            s.winrate
        FROM {ctx_table} hc
        CROSS JOIN LATERAL UNNEST(hc.ally_heroes) WITH ORDINALITY AS t(ally)
        LEFT JOIN {base_sy} s
               ON ((s.hero1 = hc.hero_id AND s.hero2 = ally)
                OR (s.hero2 = hc.hero_id AND s.hero1 = ally))
    )
    SELECT
        match_id,
        hero_id,
        AVG(winrate) AS synergy_avg,
        MAX(winrate) AS synergy_max,
        SUM(winrate) AS synergy_sum,
        SUM(CASE WHEN winrate >= 55 THEN 1 ELSE 0 END) AS synergy_strong_count
    FROM exploded
    GROUP BY match_id, hero_id;
    """
    con.execute(query)
    print(con.execute(f"SELECT COUNT(*) AS n FROM {agg_table}").df())

### Build `hero_counter_agg`

In [20]:
def build_hero_counter_agg():
    ctx_table    = "hero_context"
    base_counter = "hero_counter"
    agg_table    = "hero_counter_agg"

    print(f"Building {agg_table} ...")

    query = f"""
    CREATE OR REPLACE TABLE {agg_table} AS
    WITH exploded AS (
        SELECT
            hc.match_id,
            hc.hero_id,
            enemy_id,
            c.winrate
        FROM {ctx_table} hc
        CROSS JOIN LATERAL UNNEST(hc.enemy_heroes) WITH ORDINALITY AS t(enemy_id)
        LEFT JOIN {base_counter} c
               ON c.hero = hc.hero_id AND c.enemy = enemy_id
    )
    SELECT
        match_id,
        hero_id,
        AVG(winrate) AS counter_avg,
        MAX(winrate) AS counter_max,
        SUM(winrate) AS counter_sum,
        SUM(CASE WHEN winrate <= 45 THEN 1 ELSE 0 END) AS counter_hard_count
    FROM exploded
    GROUP BY match_id, hero_id;
    """
    con.execute(query)
    print(con.execute(f"SELECT COUNT(*) AS n FROM {agg_table}").df())

### Build `hero_lane_matchup`

In [21]:
def build_hero_lane_matchup():
    hvh_table  = "hero_vs_hero_lane"
    soul_table = "hero_soul_matchup"
    lm_table   = "hero_lane_matchup"

    print(f"Building {lm_table} ...")

    query = f"""
    CREATE OR REPLACE TABLE {lm_table} AS
    SELECT
        m.match_id,
        m.hero          AS hero_id,
        m.opponent      AS lane_opponent,
        m.advantage     AS lane_advantage,
        s.avg_soul_diff,
        s.avg_souls_raw,
        s.tower_rate    AS lane_tower_rate
    FROM {hvh_table} m
    LEFT JOIN {soul_table} s
           ON s.hero = m.hero
          AND s.opponent = m.opponent;
    """
    con.execute(query)
    print(con.execute(f"SELECT COUNT(*) AS n FROM {lm_table}").df())

### Build `hero_item_usage_phase`

In [22]:
def build_hero_item_usage_phase():
    hiu_table  = "hero_item_usage"
    hiup_table = "hero_item_usage_phase"

    print(f"Building {hiup_table} ...")

    query = f"""
    CREATE OR REPLACE TABLE {hiup_table} AS
    SELECT
        match_id,
        hero_id,
        item_id,
        CASE
            WHEN time_s < 480 THEN 'early'
            WHEN time_s < 1200 THEN 'mid'
            WHEN time_s < 2100 THEN 'late'
            ELSE 'very_late'
        END AS phase
    FROM {hiu_table};
    """
    con.execute(query)
    print(con.execute(f"SELECT COUNT(*) AS n FROM {hiup_table}").df())

## Build

In [23]:
print("\n==============================")
print(f"BUILDING PHASE 3 CONTEXT TABLES – ACTIVE PATCH = {ACTIVE_PATCH_TYPE.upper()}")
print("==============================")

if CAN_TRAIN:
    build_hero_context()
    build_hero_synergy_agg()
    build_hero_lane_matchup()
    build_hero_counter_agg()
    build_hero_item_usage_phase()
else:
    print("Skipping Phase 3 because CAN_TRAIN is False.")



BUILDING PHASE 3 CONTEXT TABLES – ACTIVE PATCH = MINOR
Building hero_context ...


        n
0  721452
Building hero_synergy_agg ...


        n
0  721452
Building hero_lane_matchup ...


         n
0  1442904
Building hero_counter_agg ...


        n
0  721452
Building hero_item_usage_phase ...


          n
0  11592438


# Phase 4
---

## Helper Functions

### Preprocessing and Sampling

In [24]:
MIN_POS_ROWS_PER_PHASE = 5_000

# -----------------------------------------------------------------------------
# Sampling helpers
# -----------------------------------------------------------------------------
def compute_pos_sample_frac(phase: str, max_rows: int) -> float:
    table = "hero_item_usage_phase"
    n_rows = con.execute(f"""
        SELECT COUNT(*) 
        FROM {table}
        WHERE phase = '{phase}'
    """).fetchone()[0]

    if n_rows == 0:
        print(f"[{phase}] 0 rows in hero_item_usage_phase.")
        return 0.0

    if n_rows < MIN_POS_ROWS_PER_PHASE:
        print(f"[{phase}] only {n_rows} rows (< {MIN_POS_ROWS_PER_PHASE}); skipping this phase.")
        return 0.0

    frac = max_rows / n_rows
    return float(min(1.0, frac))

def load_phase_positives(phase: str, max_rows: int) -> pd.DataFrame:
    """
    Sample positive rows for a given phase:
      - Each positive row = (match, hero, item, phase) where the item was bought.
      - Joins in hero context + synergy + counter + lane matchup features.

    Item winrates are merged later in Python (to keep SQL simpler).
    """
    frac = compute_pos_sample_frac(phase, max_rows)
    if frac == 0.0:
        print(f"[{phase}] No rows found.")
        return pd.DataFrame()

    pct = frac * 100.0
    print(f"[{phase}] Sampling ~{pct:.2f}% positives (target <= {max_rows})")

    ctx_table  = "hero_context"
    sy_table   = "hero_synergy_agg"
    ct_table   = "hero_counter_agg"
    lm_table   = "hero_lane_matchup"
    hiup_table = "hero_item_usage_phase"

    query = f"""
    SELECT
        hc.match_id,
        hc.hero_id,
        u.item_id,
        u.phase,
        hc.team,
        hc.won,
        hc.assigned_lane,
        hc.duration_s,
        hc.souls_9m,
        hc.cs_9m,
        hc.kills_9m,
        -- signed lane advantage (Team0 POV)
        CASE 
            WHEN hc.team = 'Team0' THEN hc.lane_advantage_score
            WHEN hc.team = 'Team1' THEN -hc.lane_advantage_score
            ELSE 0
        END AS lane_adv_signed,
        hc.team0_tier,
        hc.team1_tier,
        COALESCE(sy.synergy_avg, 50.0)          AS synergy_avg,
        COALESCE(sy.synergy_max, 50.0)          AS synergy_max,
        COALESCE(sy.synergy_sum, 0.0)           AS synergy_sum,
        COALESCE(sy.synergy_strong_count, 0.0)  AS synergy_strong_count,
        COALESCE(ct.counter_avg, 50.0)          AS counter_avg,
        COALESCE(ct.counter_max, 50.0)          AS counter_max,
        COALESCE(ct.counter_sum, 0.0)           AS counter_sum,
        COALESCE(ct.counter_hard_count, 0.0)    AS counter_hard_count,
        COALESCE(lm.lane_opponent, -1)          AS lane_opponent,
        COALESCE(lm.avg_soul_diff, 0.0)         AS avg_soul_diff,
        COALESCE(lm.avg_souls_raw, 0.0)         AS avg_souls_raw,
        COALESCE(lm.lane_tower_rate, 0.5)       AS lane_tower_rate
    FROM {hiup_table} u
    JOIN {ctx_table} hc
      ON hc.match_id = u.match_id
     AND hc.hero_id  = u.hero_id
    LEFT JOIN {sy_table} sy
      ON sy.match_id = hc.match_id
     AND sy.hero_id  = hc.hero_id
    LEFT JOIN {ct_table} ct
      ON ct.match_id = hc.match_id
     AND ct.hero_id  = hc.hero_id
    LEFT JOIN {lm_table} lm
      ON lm.match_id = hc.match_id
     AND lm.hero_id  = hc.hero_id
    WHERE u.phase = '{phase}'
    USING SAMPLE {pct} PERCENT (bernoulli);
    """

    df = con.execute(query).df()
    print(f"[{phase}] Loaded {len(df)} positive rows.")
    return df


### Load Item Winrates

In [25]:
def load_item_stats():
    hiw_table = "hero_item_winrate"

    # Item global winrate (averaged over heroes)
    item_global_df = con.execute(f"""
        SELECT
            item_id,
            AVG(smoothed_wr) AS item_global_wr
        FROM {hiw_table}
        GROUP BY item_id;
    """).df()

    # Hero-specific item winrate (smoothed)
    hero_item_df = con.execute(f"""
        SELECT
            hero_id,
            item_id,
            smoothed_wr AS hero_item_wr
        FROM {hiw_table};
    """).df()

    return item_global_df, hero_item_df

### Generate Negative Samples

In [ ]:
# -----------------------------------------------------------------------------
# Negative sampling: given positive rows, generate random unseen items
# -----------------------------------------------------------------------------
def generate_negatives(
    df_pos: pd.DataFrame,
    all_items: np.ndarray,
    n_neg_per_pos: int = NEG_PER_POS,
    rng: np.random.Generator | None = None,
) -> pd.DataFrame:
    """
    For each positive row (match, hero, phase, item_id),
    sample n_neg_per_pos other items that hero did not buy in that match+phase.

    We copy the context features and only change item_id and item_bought=0.
    """
    if df_pos.empty:
        return pd.DataFrame(columns=df_pos.columns)

    if rng is None:
        rng = np.random.default_rng(RANDOM_SEED)

    # For each (match, hero, phase) track which items were actually bought
    pos_items = {}
    for (m, h, p, it) in zip(
        df_pos["match_id"],
        df_pos["hero_id"],
        df_pos["phase"],
        df_pos["item_id"],
    ):
        key = (int(m), int(h), str(p))
        pos_items.setdefault(key, set()).add(int(it))

    n_items = len(all_items)
    ctx_keys = list(zip(df_pos["match_id"], df_pos["hero_id"], df_pos["phase"]))

    neg_rows_idx = []
    neg_item_ids = []

    for idx, (m, h, p) in enumerate(ctx_keys):
        key = (int(m), int(h), str(p))
        bought = pos_items.get(key, set())

        for _ in range(n_neg_per_pos):
            item = None
            for _attempt in range(5):
                cand = int(all_items[rng.integers(n_items)])
                if cand not in bought:
                    item = cand
                    break
            if item is None:
                item = int(all_items[rng.integers(n_items)])

            neg_rows_idx.append(idx)
            neg_item_ids.append(item)

    if not neg_rows_idx:
        return pd.DataFrame(columns=df_pos.columns)

    neg_df = df_pos.iloc[neg_rows_idx].copy()
    neg_df["item_id"] = np.array(neg_item_ids, dtype=np.int64)
    neg_df["item_bought"] = 0
    return neg_df


### Preprocessing and Train/Valid Split

In [ ]:
def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply consistent preprocessing:
      - Ensure label is int 0/1.
      - Encode team, lane, hero_id, item_id, lane_opponent as ints.
      - Cast numeric features to float32.
      - Drop rows with NaNs in used columns.
      - Keep only metadata + features + label.
    """
    if df.empty:
        return df

    # Ensure label exists and is numeric 0/1
    if LABEL_COL not in df.columns:
        raise ValueError(f"Label column {LABEL_COL} missing.")
    df[LABEL_COL] = df[LABEL_COL].astype(int)

    # Encode team
    if "team" in df.columns:
        df["team"] = df["team"].map({"Team0": 0, "Team1": 1}).astype("int8")

    # Lane
    if "assigned_lane" in df.columns:
        df["assigned_lane"] = df["assigned_lane"].astype("int8")

    # Hero/item IDs
    if "hero_id" in df.columns:
        df["hero_id"] = df["hero_id"].astype("int16")
    if "item_id" in df.columns:
        df["item_id"] = df["item_id"].astype("int32")
    if "lane_opponent" in df.columns:
        df["lane_opponent"] = df["lane_opponent"].fillna(-1).astype("int16")

    # Numeric features 
    for col in NUMERIC_FEATURES:
        if col in df.columns:
            df[col] = df[col].astype("float32")

    # Drop rows with NaNs in any used feature or label
    cols_needed = [c for c in ALL_FEATURES + [LABEL_COL] if c in df.columns]
    df = df.dropna(subset=cols_needed)

    # Keep only metadata + features + label
    keep_cols = list(set(METADATA_COLS + ALL_FEATURES + [LABEL_COL]))
    keep_cols = [c for c in keep_cols if c in df.columns]
    df = df[keep_cols]

    return df


def train_valid_split(df: pd.DataFrame, valid_frac: float = 0.1, seed: int = RANDOM_SEED):
    """
    Split into train/valid, stratified on LABEL_COL when possible.
    """
    train_df, valid_df = train_test_split(
        df,
        test_size=valid_frac,
        random_state=seed,
        shuffle=True,
        stratify=df[[LABEL_COL]] if df[LABEL_COL].nunique() > 1 else None,
    )
    return train_df.reset_index(drop=True), valid_df.reset_index(drop=True)


def split_X_y(df: pd.DataFrame):
    """
    Separate features X, labels y, and metadata (match_id, hero_id, item_id, phase).
    """
    X = df[ALL_FEATURES].copy()
    y = df[LABEL_COL].copy()
    meta = df[[c for c in METADATA_COLS if c in df.columns]].copy()
    return X, y, meta


def make_scale_pos_weight(y: pd.Series) -> float:
    """
    Compute scale_pos_weight = (#neg / #pos), used by XGBoost.
    """
    n_pos = (y == 1).sum()
    n_neg = (y == 0).sum()
    if n_pos == 0:
        return 1.0
    return float(n_neg) / float(n_pos)


def build_model(scale_pos_weight: float) -> XGBClassifier:
    """
    Build an XGBClassifier configured for this dataset.
    On your training machine: device='cuda'.
    On Lambda: you can reload the model and run on CPU, it’s fine.
    """
    params = dict(
        n_estimators=400,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",
        eval_metric="logloss",
        tree_method="hist",
        device="cuda",
        scale_pos_weight=scale_pos_weight,
        reg_lambda=1.0,
        random_state=RANDOM_SEED,
    )
    model = XGBClassifier(**params)
    return model


### Globals

In [28]:
# -----------------------------------------------------------------------------
# Pre-load item stats for the ACTIVE PATCH (single set of tables)
# -----------------------------------------------------------------------------
item_global_stats, hero_item_stats = load_item_stats()

def merge_item_stats(df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge item_global_wr and hero_item_wr into df based on item_id, hero_id.
    """
    if df.empty:
        return df

    ig = item_global_stats
    hi = hero_item_stats

    df = df.merge(ig, on="item_id", how="left")
    df = df.merge(hi, on=["hero_id", "item_id"], how="left")

    df["item_global_wr"] = df["item_global_wr"].fillna(50.0)
    df["hero_item_wr"] = df["hero_item_wr"].fillna(df["item_global_wr"])

    return df

## Training

In [29]:
# -----------------------------------------------------------------------------
# All items universe (for negative sampling)
# -----------------------------------------------------------------------------
all_items = con.execute("SELECT id FROM shop_items").df()["id"].to_numpy()

# -----------------------------------------------------------------------------
# TRAIN PHASE MODELS (early/mid/late/very_late)
# -----------------------------------------------------------------------------
if not CAN_TRAIN:
    print("CAN_TRAIN is False – skipping model training for this run.")
    metrics = []
else:
    metrics = []

    for phase in PHASES:
        print("\n" + "=" * 80)
        print(f"TRAINING PHASE MODEL: {phase.upper()}")
        print("=" * 80)

        rng = np.random.default_rng(RANDOM_SEED)

        # 1) Load positives from active patch window
        df_pos = load_phase_positives(
            phase=phase,
            max_rows=MAX_POS_ROWS[ACTIVE_PATCH_TYPE],
        )
        if df_pos.empty:
            print(f"[{phase}] No positives for ACTIVE PATCH, skipping this phase.")
            continue

        # 2) Generate negatives
        df_neg = generate_negatives(df_pos, all_items, NEG_PER_POS, rng)

        # 3) Label positives as item_bought=1
        df_pos["item_bought"] = 1

        # 4) Combine pos + neg
        df_all = pd.concat([df_pos, df_neg], ignore_index=True)
        print(f"[{phase}] Combined pos+neg rows: {len(df_all)}")

        # 5) Merge item stats
        df_all = merge_item_stats(df_all)

        # 6) Preprocess
        df_all = preprocess_df(df_all)
        print(f"[{phase}] Rows after preprocessing: {len(df_all)}")

        if df_all[LABEL_COL].nunique() < 2:
            print(f"[{phase}] Not enough label variety, skipping training.")
            continue

        # 7) Train/valid split
        train_df, valid_df = train_valid_split(df_all, valid_frac=0.1)

        X_train, y_train, meta_train = split_X_y(train_df)
        X_valid, y_valid, meta_valid = split_X_y(valid_df)

        print(f"[{phase}] Train size: {len(X_train)}, Valid size: {len(X_valid)}")

        # 8) Build + train model
        spw = make_scale_pos_weight(y_train)
        print(f"[{phase}] scale_pos_weight = {spw:.3f}")
        model = build_model(spw)

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_valid, y_valid)],
            verbose=True,
        )

        # 9) Evaluate
        valid_pred = model.predict_proba(X_valid)[:, 1]
        valid_auc = roc_auc_score(y_valid, valid_pred)
        valid_ll = log_loss(y_valid, valid_pred)

        print(f"[{phase}] Valid AUC = {valid_auc:.4f}, LogLoss = {valid_ll:.4f}")

        # 10) Save model
        model_path = MODELS_DIR / f"{phase}.json"
        model.get_booster().save_model(model_path.as_posix())
        print(f"[{phase}] Saved model to: {model_path}")

        # 11) Store metrics
        metrics.append({
            "phase": phase,
            "valid_auc": float(valid_auc),
            "valid_logloss": float(valid_ll),
            "scale_pos_weight": float(spw),
            "n_train": int(len(X_train)),
            "n_valid": int(len(X_valid)),
            "n_rows_total": int(len(df_all)),
            "patch_type": ACTIVE_PATCH_TYPE,
            "patch_start_ts": str(ACTIVE_PATCH_START),
        })

metrics_df = pd.DataFrame(metrics)
metrics_df


TRAINING PHASE MODEL: EARLY
[early] Sampling ~26.71% positives (target <= 800000)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[early] Loaded 1601247 positive rows.


[early] Combined pos+neg rows: 6404988


[early] Rows after preprocessing: 6404988


...

[398]	validation_0-logloss:0.42366


[399]	validation_0-logloss:0.42362


[very_late] Valid AUC = 0.8904, LogLoss = 0.4236


[very_late] Saved model to: /home/karan/dev/deadlock/deadlock-backend/models/minor_20251229/very_late.json


,phase,valid_auc,valid_logloss,scale_pos_weight,n_train,n_valid,n_rows_total,patch_type,patch_start_ts
0,early,0.963521,0.253544,3.000001,5764489,640499,6404988,minor,2025-12-29 00:00:00+00:00
1,mid,0.909179,0.393325,2.999999,5755719,639525,6395244,minor,2025-12-29 00:00:00+00:00
2,late,0.884148,0.437833,3.000000,5760428,640048,6400476,minor,2025-12-29 00:00:00+00:00
3,very_late,0.890424,0.423616,3.000002,2505369,278375,2783744,minor,2025-12-29 00:00:00+00:00


## Metadata

In [30]:
import json

metadata = {
    "model_version": MODEL_VERSION,
    "patch_type": ACTIVE_PATCH_TYPE,
    "patch_start_ts": str(ACTIVE_PATCH_START),
    "phases": PHASES,
    "features": ALL_FEATURES,
    "numeric_features": NUMERIC_FEATURES,
    "categorical_features": CATEGORICAL_FEATURES,
    "metrics": metrics,
}

meta_path = MODELS_DIR / "training_metadata.json"
with open(meta_path, "w") as f:
    json.dump(metadata, f, indent=2)

print(f"Saved training metadata → {meta_path}")
metrics_df

Saved training metadata → /home/karan/dev/deadlock/deadlock-backend/models/minor_20251229/training_metadata.json


,phase,valid_auc,valid_logloss,scale_pos_weight,n_train,n_valid,n_rows_total,patch_type,patch_start_ts
0,early,0.963521,0.253544,3.000001,5764489,640499,6404988,minor,2025-12-29 00:00:00+00:00
1,mid,0.909179,0.393325,2.999999,5755719,639525,6395244,minor,2025-12-29 00:00:00+00:00
2,late,0.884148,0.437833,3.000000,5760428,640048,6400476,minor,2025-12-29 00:00:00+00:00
3,very_late,0.890424,0.423616,3.000002,2505369,278375,2783744,minor,2025-12-29 00:00:00+00:00


## Inference

### Item Transitions

In [31]:
from collections import defaultdict

TRANSITION_TABLE = "item_transition_stats"

_transition_df = con.execute(f"""
    SELECT hero_id, item_current, item_next, trans_prob
    FROM {TRANSITION_TABLE}
""").df()

_global_transition_df = con.execute(f"""
    SELECT 
        item_current,
        item_next,
        SUM(trans_count) AS trans_count
    FROM {TRANSITION_TABLE}
    GROUP BY item_current, item_next
""").df()

def _build_hero_transition_dict(df: pd.DataFrame):
    """
    nested dict: hero -> item_current -> {item_next -> prob}
    """
    trans = defaultdict(lambda: defaultdict(dict))
    for _, row in df.iterrows():
        h = int(row["hero_id"])
        cur = int(row["item_current"])
        nxt = int(row["item_next"])
        p = float(row["trans_prob"])
        trans[h][cur][nxt] = p
    return trans

def _build_global_transition_dict(df: pd.DataFrame):
    """
    dict: item_current -> {item_next -> prob} using global counts.
    """
    g = defaultdict(dict)
    # normalize per item_current
    grouped = df.groupby("item_current")
    for cur, grp in grouped:
        total = grp["trans_count"].sum()
        for _, row in grp.iterrows():
            nxt = int(row["item_next"])
            p = float(row["trans_count"]) / float(total)
            g[int(cur)][nxt] = p
    return g

TRANS_HERO = _build_hero_transition_dict(_transition_df)
TRANS_GLOBAL = _build_global_transition_dict(_global_transition_df)

def get_transition_prob(hero_id: int, item_current: int, item_next: int) -> float:
    hero_map = TRANS_HERO.get(int(hero_id), {})
    if item_current in hero_map and item_next in hero_map[item_current]:
        return hero_map[item_current][item_next]

    global_map = TRANS_GLOBAL.get(int(item_current), {})
    if item_next in global_map:
        return global_map[item_next]

    return 0.01

### Build `inference_context`

In [ ]:
import xgboost as xgb

# =============================================================================
# INFERENCE HELPERS
# =============================================================================
# We use ACTIVE PATCH tables as the basis for inference (consistent with training).
# =============================================================================

# Global averages from match_info for duration and rank tiers
_global_match_stats = con.execute("""
    SELECT 
        AVG(duration_s)  AS avg_duration_s,
        AVG(team0_tier)  AS avg_team0_tier,
        AVG(team1_tier)  AS avg_team1_tier
    FROM match_info
""").df().iloc[0].to_dict()

def _safe_get(val, default):
    return default if val is None or pd.isna(val) else val


def build_inference_context_row(
    hero_id: int,
    lane_ally_id: int,
    team_other_ids: list[int],
    lane_enemy_ids: list[int],
    enemy_other_ids: list[int],
) -> pd.DataFrame:
    """
    Build a single hero-context row for inference from the exact draft the user inputs.

    Inputs:
      - hero_id: the hero we are recommending items for (user hero)
      - lane_ally_id: hero in same lane on user's team
      - team_other_ids: remaining 4 allies
      - lane_enemy_ids: 2 heroes in enemy lane
      - enemy_other_ids: remaining 4 enemies

    Returns:
      - A DataFrame with 1 row and all non-item features filled:
        duration_s, souls_9m, cs_9m, kills_9m, lane_adv_signed(≈0),
        team0_tier, team1_tier, synergy_*, counter_*, lane_opponent,
        avg_soul_diff, avg_souls_raw, lane_tower_rate, etc.
    """
    assert len(team_other_ids) == 4, "Expected 4 non-lane allies."
    assert len(lane_enemy_ids) == 2, "Expected 2 lane enemies."
    assert len(enemy_other_ids) == 4, "Expected 4 non-lane enemies."

    sy_table    = "hero_synergy"
    counter_tbl = "hero_counter"
    soul_tbl    = "hero_soul_matchup"
    snap_tbl    = "hero_lane_snap_9"

    # -------------------------------------------------------------------------
    # 1) Basic lane performance stats for this hero (averaged over historical games)
    # -------------------------------------------------------------------------
    snap_row = con.execute(
        f"""
        SELECT 
            AVG(souls_9m) AS souls_9m,
            AVG(cs_9m)    AS cs_9m,
            AVG(kills_9m) AS kills_9m
        FROM {snap_tbl}
        WHERE hero_id = ?
        """,
        [hero_id],
    ).df().iloc[0]

    souls_9m = _safe_get(snap_row["souls_9m"], 0.0)
    cs_9m    = _safe_get(snap_row["cs_9m"], 0.0)
    kills_9m = _safe_get(snap_row["kills_9m"], 0.0)

    # -------------------------------------------------------------------------
    # 2) Synergy features vs allies (same logic as hero_synergy_agg, but direct)
    # -------------------------------------------------------------------------
    ally_ids = [lane_ally_id] + team_other_ids

    placeholders = ",".join(["?"] * len(ally_ids))
    sy_df = con.execute(
        f"""
        SELECT hero1, hero2, winrate
        FROM {sy_table}
        WHERE (hero1 = ? AND hero2 IN ({placeholders}))
           OR (hero2 = ? AND hero1 IN ({placeholders}))
        """,
        [hero_id, *ally_ids, hero_id, *ally_ids],
    ).df()

    # Build mapping ally_id -> winrate
    sy_map = {}
    for _, row in sy_df.iterrows():
        h1, h2, wr = int(row["hero1"]), int(row["hero2"]), float(row["winrate"])
        other = h2 if h1 == hero_id else h1
        sy_map[other] = wr

    # Fill in missing ally pairs with neutral 50% winrate
    sy_vals = []
    for aid in ally_ids:
        sy_vals.append(sy_map.get(aid, 50.0))

    if len(sy_vals) == 0:
        sy_vals = [50.0]

    synergy_avg = float(np.mean(sy_vals))
    synergy_max = float(np.max(sy_vals))
    synergy_sum = float(np.sum(sy_vals))
    synergy_strong_count = float(np.sum(np.array(sy_vals) >= 55.0))

    # -------------------------------------------------------------------------
    # 3) Counter features vs enemies (same logic as hero_counter_agg)
    # -------------------------------------------------------------------------
    enemy_ids = lane_enemy_ids + enemy_other_ids
    placeholders = ",".join(["?"] * len(enemy_ids))

    ct_df = con.execute(
        f"""
        SELECT enemy, winrate
        FROM {counter_tbl}
        WHERE hero = ? AND enemy IN ({placeholders})
        """,
        [hero_id, *enemy_ids],
    ).df()

    ct_map = {int(r["enemy"]): float(r["winrate"]) for _, r in ct_df.iterrows()}

    ct_vals = []
    for eid in enemy_ids:
        ct_vals.append(ct_map.get(eid, 50.0))

    if len(ct_vals) == 0:
        ct_vals = [50.0]

    counter_avg = float(np.mean(ct_vals))
    counter_max = float(np.max(ct_vals))
    counter_sum = float(np.sum(ct_vals))
    counter_hard_count = float(np.sum(np.array(ct_vals) <= 45.0))

    # -------------------------------------------------------------------------
    # 4) Lane matchup metrics (hero vs lane enemies) from hero_soul_matchup
    # -------------------------------------------------------------------------
    placeholders_lane = ",".join(["?"] * len(lane_enemy_ids))
    lane_df = con.execute(
        f"""
        SELECT opponent, avg_soul_diff, avg_souls_raw, tower_rate
        FROM {soul_tbl}
        WHERE hero = ? AND opponent IN ({placeholders_lane})
        """,
        [hero_id, *lane_enemy_ids],
    ).df()

    if len(lane_df) == 0:
        lane_opponent   = -1
        avg_soul_diff   = 0.0
        avg_souls_raw   = 0.0
        lane_tower_rate = 0.5
    else:
        worst_idx = lane_df["avg_soul_diff"].idxmin()
        lane_opponent = int(lane_df.loc[worst_idx, "opponent"])

        avg_soul_diff   = float(lane_df["avg_soul_diff"].mean())
        avg_souls_raw   = float(lane_df["avg_souls_raw"].mean())
        lane_tower_rate = float(lane_df["tower_rate"].mean())

    # -------------------------------------------------------------------------
    # 5) Assemble context row
    # -------------------------------------------------------------------------
    duration_s = _safe_get(_global_match_stats["avg_duration_s"], 1800.0)
    team0_tier = _safe_get(_global_match_stats["avg_team0_tier"], PHANTOM_TIER)
    team1_tier = _safe_get(_global_match_stats["avg_team1_tier"], PHANTOM_TIER)

    ctx = {
        # Metadata
        "match_id": 0,
        "phase": "unknown",
        "hero_id": int(hero_id),
        "item_id": -1,           # placeholder
        "team": "Team0",         # assume our hero is Team0
        "assigned_lane": 1,      # arbitrary

        # Hero / match context
        "duration_s": float(duration_s),
        "souls_9m": float(souls_9m),
        "cs_9m": float(cs_9m),
        "kills_9m": float(kills_9m),
        "lane_adv_signed": 0.0,  # neutral lane advantage at prediction time

        "team0_tier": float(team0_tier),
        "team1_tier": float(team1_tier),

        # Synergy summary
        "synergy_avg": synergy_avg,
        "synergy_max": synergy_max,
        "synergy_sum": synergy_sum,
        "synergy_strong_count": synergy_strong_count,

        # Counter summary
        "counter_avg": counter_avg,
        "counter_max": counter_max,
        "counter_sum": counter_sum,
        "counter_hard_count": counter_hard_count,

        # Lane matchup (vs primary lane opponent)
        "lane_opponent": lane_opponent,
        "avg_soul_diff": avg_soul_diff,
        "avg_souls_raw": avg_souls_raw,
        "lane_tower_rate": lane_tower_rate,
    }

    return pd.DataFrame([ctx])

### Preprocessing Inference

In [ ]:
def preprocess_for_inference(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess a feature DataFrame for inference:
      - Encode team, lane, hero_id, item_id, lane_opponent.
      - Cast numeric features to float32.
      - Keep only ALL_FEATURES.
    """
    df = df.copy()

    # Team encoding
    if "team" in df.columns:
        df["team"] = df["team"].map({"Team0": 0, "Team1": 1}).astype("int8")

    # Assigned lane
    if "assigned_lane" in df.columns:
        df["assigned_lane"] = df["assigned_lane"].astype("int8")

    # Hero / item / lane_opponent
    if "hero_id" in df.columns:
        df["hero_id"] = df["hero_id"].astype("int16")
    if "item_id" in df.columns:
        df["item_id"] = df["item_id"].astype("int32")
    if "lane_opponent" in df.columns:
        df["lane_opponent"] = df["lane_opponent"].fillna(-1).astype("int16")

    # Numeric features
    for col in NUMERIC_FEATURES:
        if col in df.columns:
            df[col] = df[col].astype("float32")

    # Keep only feature columns
    # Fill any missing with 0 for safety
    df = df[[c for c in ALL_FEATURES if c in df.columns]].fillna(0.0)

    return df

### Scoring Items

In [ ]:
def score_items_all_phases(
    models: dict[str, XGBClassifier],
    hero_id: int,
    lane_ally_id: int,
    team_other_ids: list[int],
    lane_enemy_ids: list[int],
    enemy_other_ids: list[int],
) -> pd.DataFrame:
    """
    Compute, for EVERY shop item, the probability of being bought in each phase,
    given the draft context.

    Returns a DataFrame with columns:
      item_id, item_name, tier, cost, shop_image, shop_image_webp, item_slot_type
      score_early, score_mid, score_late, score_very_late
    """
    # 1) Single context row for this hero + draft
    ctx_df = build_inference_context_row(
        hero_id=hero_id,
        lane_ally_id=lane_ally_id,
        team_other_ids=team_other_ids,
        lane_enemy_ids=lane_enemy_ids,
        enemy_other_ids=enemy_other_ids,
    )

    # 2) Item metadata + images
    items_df = con.execute("""
        SELECT 
            si.id   AS item_id,
            si.name AS item_name,
            si.tier,
            si.cost,
            ia.shop_image,
            ia.shop_image_webp,
            ia.item_slot_type
        FROM shop_items si
        LEFT JOIN item_assets ia
               ON ia.item_id = si.id
    """).df()

    # 3) Repeat context for each item
    base = ctx_df.loc[ctx_df.index.repeat(len(items_df))].reset_index(drop=True)
    base["item_id"] = items_df["item_id"].values

    # 4) Merge item stats and preprocess ONCE
    base = merge_item_stats(base)
    X_base = preprocess_for_inference(base)

    # 5) Score per phase
    for phase in PHASES:
        model = models[phase]
        scores = model.predict_proba(X_base)[:, 1]
        items_df[f"score_{phase}"] = scores

    return items_df

### Build Sequence Transitions

In [ ]:
def build_sequence_with_transitions(
    hero_id: int,
    items_df: pd.DataFrame,
    slots_per_phase: dict[str, int] | None = None,
    lambda_trans: float = 1.0,
    candidate_top_n: int = 50,
) -> dict[str, list[dict]]:
    """
    Greedy sequence builder:
      - Uses per-phase scores from items_df (score_early, score_mid, ...)
      - Uses item_transition_stats to bias toward realistic upgrade paths.
      - Produces a global ordered sequence of items, then splits into phases.

    slots_per_phase: how many items to suggest per phase.
    lambda_trans: strength of transition coherence vs model score.
    candidate_top_n: per step, only consider top-N items by phase score
                     to keep it cheap and avoid junk.
    """
    if slots_per_phase is None:
        slots_per_phase = {
            "early": 4,
            "mid": 4,
            "late": 4,
            "very_late": 3,
        }

    phase_order: list[str] = []
    for ph in PHASES:
        phase_order.extend([ph] * slots_per_phase[ph])

    chosen_ids: set[int] = set()
    sequence: list[int] = []

    for step, phase in enumerate(phase_order):
        score_col = f"score_{phase}"
        cand = items_df[~items_df["item_id"].isin(chosen_ids)].copy()
        if score_col not in cand.columns:
            continue
        cand = cand.sort_values(score_col, ascending=False).head(candidate_top_n)

        if cand.empty:
            break

        best_item = None
        best_score = -1e9

        for _, row in cand.iterrows():
            item_id = int(row["item_id"])
            base_score = float(row[score_col])

            if base_score <= 0:
                continue

            log_base = float(np.log(base_score + 1e-8))

            if not sequence:
                log_trans = 0.0
            else:
                prev_item = sequence[-1]
                p_trans = get_transition_prob(hero_id, prev_item, item_id)
                log_trans = float(np.log(p_trans + 1e-6))

            total_score = log_base + lambda_trans * log_trans

            if total_score > best_score:
                best_score = total_score
                best_item = item_id

        if best_item is None:
            break

        sequence.append(best_item)
        chosen_ids.add(best_item)

    # ---------------------------------------------------------------------
    # Split sequence into phases and pack with metadata
    # ---------------------------------------------------------------------
    results = {ph: [] for ph in PHASES}
    idx = 0
    for ph in PHASES:
        slots = slots_per_phase[ph]
        score_col = f"score_{ph}"
        for _ in range(slots):
            if idx >= len(sequence):
                break
            iid = sequence[idx]
            row = items_df[items_df["item_id"] == iid].iloc[0]
            results[ph].append({
                "item_id": int(row["item_id"]),
                "name": row["item_name"],
                "tier": int(_safe_get(row.get("tier"), 0)),
                "cost": int(_safe_get(row.get("cost"), 0)),
                "score": float(_safe_get(row.get(score_col), 0.0)),
                "shop_image": row.get("shop_image"),
                "shop_image_webp": row.get("shop_image_webp"),
                "item_slot_type": row.get("item_slot_type"),
            })
            idx += 1

    return results

### Loading Phase Models

In [ ]:
def load_phase_models(models_dir: Path = MODELS_DIR) -> dict[str, XGBClassifier]:
    """
    Load XGBoost models for each phase from disk.
    Assumes files: models_dir/early.json, mid.json, late.json, very_late.json
    """
    models = {}
    for phase in PHASES:
        model_path = models_dir / f"{phase}.json"
        model = XGBClassifier()
        model.load_model(model_path.as_posix())
        models[phase] = model
    return models


### Recommending Items For Phase

In [ ]:
def recommend_items_for_phase(
    models: dict[str, XGBClassifier],
    phase: str,
    hero_id: int,
    lane_ally_id: int,
    team_other_ids: list[int],
    lane_enemy_ids: list[int],
    enemy_other_ids: list[int],
    top_k: int = 10,
    lambda_trans: float = 0.3,
    candidate_top_n: int = 30,
) -> list[dict]:
    model = models[phase]

    # 1) Context row
    ctx_df = build_inference_context_row(
        hero_id=hero_id,
        lane_ally_id=lane_ally_id,
        team_other_ids=team_other_ids,
        lane_enemy_ids=lane_enemy_ids,
        enemy_other_ids=enemy_other_ids,
    )

    # 2) All items + assets
    items_df = con.execute("""
        SELECT 
            si.id   AS item_id,
            si.name AS item_name,
            si.tier,
            si.cost,
            ia.shop_image,
            ia.shop_image_webp,
            ia.item_slot_type
        FROM shop_items si
        LEFT JOIN item_assets ia
               ON ia.item_id = si.id
    """).df()

    base = ctx_df.loc[ctx_df.index.repeat(len(items_df))].reset_index(drop=True)
    base["item_id"] = items_df["item_id"].values

    base = merge_item_stats(base)
    X = preprocess_for_inference(base)

    # Phase-only model score
    items_df["score_phase"] = models[phase].predict_proba(X)[:, 1]

    # 3) Use your transition-aware builder
    build = build_sequence_with_transitions(
        hero_id=hero_id,
        items_df=items_df,
        lambda_trans=lambda_trans,
        candidate_top_n=candidate_top_n,
        slots_per_phase=SLOTS_PER_PHASE_DEFAULT,
    )

    phase_items = build.get(phase, [])

    # Fallback bare model top-k if transition logic yields nothing
    if not phase_items:
        fallback_top = (
            items_df.sort_values("score_phase", ascending=False)
                    .head(top_k)
        )
        phase_items = [
            {
                "item_id": int(r["item_id"]),
                "name": r["item_name"],
                "tier": int(_safe_get(r.get("tier"), 0)),
                "cost": int(_safe_get(r.get("cost"), 0)),
                "score": float(r["score_phase"]),
                "shop_image": r.get("shop_image"),
                "shop_image_webp": r.get("shop_image_webp"),
                "item_slot_type": r.get("item_slot_type"),
            }
            for _, r in fallback_top.iterrows()
        ]

    return phase_items[:top_k]

### Recommend Build Function

In [38]:
def recommend_build(
    hero_id: int,
    lane_ally_id: int,
    team_other_ids: list[int],
    lane_enemy_ids: list[int],
    enemy_other_ids: list[int],
    top_k_per_phase: int = 10,
    lambda_by_phase: dict[str, float] = LAMBDA_BY_PHASE_DEFAULT,
    candidate_top_n: int = TOP_N_DEFAULT,
) -> dict[str, list[dict]]:
    models = load_phase_models(MODELS_DIR)

    recs: dict[str, list[dict]] = {}
    for phase in PHASES:
        lam = lambda_by_phase.get(phase, 0.3)

        recs[phase] = recommend_items_for_phase(
            models=models,
            phase=phase,
            hero_id=hero_id,
            lane_ally_id=lane_ally_id,
            team_other_ids=team_other_ids,
            lane_enemy_ids=lane_enemy_ids,
            enemy_other_ids=enemy_other_ids,
            top_k=top_k_per_phase,
            lambda_trans=lam,
            candidate_top_n=candidate_top_n,
        )

    return recs

### Testing Code

In [39]:
# ===========================================
# TEST: draft-sensitivity of recommend_build
# ===========================================
heroes_df = con.execute("SELECT id, name FROM heroes ORDER BY id").df()
all_heroes = heroes_df["id"].tolist()
id_to_name = dict(zip(heroes_df["id"], heroes_df["name"]))

rng = np.random.default_rng(123)

def random_draft_for_hero(hero_id: int):
    """
    Sample a random 10-hero draft around a fixed hero_id.
    Returns: lane_ally_id, team_other_ids, lane_enemy_ids, enemy_other_ids
    """
    others = [h for h in all_heroes if h != hero_id]
    rng.shuffle(others)

    lane_ally_id  = others[0]
    team_other_ids = others[1:5]   # 4 remaining allies
    lane_enemy_ids = others[5:7]   # 2 lane enemies
    enemy_other_ids = others[7:11] # 4 remaining enemies

    return lane_ally_id, team_other_ids, lane_enemy_ids, enemy_other_ids

def build_item_set(build: dict) -> set[int]:
    """
    Flatten all phases into a single set of item_ids.
    """
    s = set()
    for phase in PHASES:
        for item in build.get(phase, []):
            s.add(item["item_id"])
    return s

def phase_item_set(build: dict, phase: str) -> set[int]:
    return set(item["item_id"] for item in build.get(phase, []))

def jaccard(a: set[int], b: set[int]) -> float:
    if not a and not b:
        return 1.0
    return len(a & b) / max(1, len(a | b))

def test_draft_sensitivity(hero_id: int, n_samples: int = 20):
    print(f"\n=== Draft sensitivity test for hero {hero_id} ({id_to_name.get(hero_id, '?')}) ===")

    # baseline random draft
    base_lane_ally, base_team_others, base_lane_enemies, base_enemy_others = random_draft_for_hero(hero_id)
    base_build = recommend_build(
        hero_id=hero_id,
        lane_ally_id=base_lane_ally,
        team_other_ids=base_team_others,
        lane_enemy_ids=base_lane_enemies,
        enemy_other_ids=base_enemy_others,
        top_k_per_phase=8,
    )

    base_all = build_item_set(base_build)

    per_phase_sims = {ph: [] for ph in PHASES}
    overall_sims = []

    for i in range(n_samples):
        la, to, le, eo = random_draft_for_hero(hero_id)
        b = recommend_build(
            hero_id=hero_id,
            lane_ally_id=la,
            team_other_ids=to,
            lane_enemy_ids=le,
            enemy_other_ids=eo,
            top_k_per_phase=8,
        )

        cur_all = build_item_set(b)
        overall_sims.append(jaccard(base_all, cur_all))

        for ph in PHASES:
            s1 = phase_item_set(base_build, ph)
            s2 = phase_item_set(b, ph)
            per_phase_sims[ph].append(jaccard(s1, s2))

    print(f"Average overall Jaccard over {n_samples} random drafts: {np.mean(overall_sims):.3f}")
    for ph in PHASES:
        print(f"  {ph:10s}: avg Jaccard = {np.mean(per_phase_sims[ph]):.3f}")

    return {
        "overall": overall_sims,
        "per_phase": per_phase_sims,
        "base_build": base_build,
    }